In [ ]:
!pip install git+https://github.com/philferriere/cocoapi.git#subdirectory=PythonAPI


  Cloning https://github.com/philferriere/cocoapi.git to c:\users\utilizador\appdata\local\temp\pip-req-build-yh1gskmk
  Resolved https://github.com/philferriere/cocoapi.git to commit 2929bd2ef6b451054755dfd7ceb09278f935f7ad


  Running command git clone -q https://github.com/philferriere/cocoapi.git 'C:\Users\Utilizador\AppData\Local\Temp\pip-req-build-yh1gskmk'


  Created wheel for pycocotools: filename=pycocotools-2.0-cp39-cp39-win_amd64.whl size=81991 sha256=2bb632cc2c2b576b4beaca5d6de5a363083304508965cc6df30b271d278e15de
  Stored in directory: C:\Users\Utilizador\AppData\Local\Temp\pip-ephem-wheel-cache-zv44n6yy\wheels\8d\25\59\5840f59a3c30bac7223e8d0f1dcf19ec4d3c249d7659d9782f
Successfully built pycocotools


In [5]:
import pandas as pd
import numpy as np

import utils.print as print_f

from utils.coco_eval import get_eval_params_dict
from utils.engine import xami_train_one_epoch, xami_evaluate, get_iou_types
from utils.plot import plot_losses, plot_ap_ars
from utils.save import get_data_from_metric_logger
from utils.coco_utils import get_cocos

from models.setup import ModelSetup
from models.build import create_multimodal_rcnn_model
from models.train import TrainingInfo
from utils.save import check_best, end_train
from data.load import get_datasets, get_dataloaders
from IPython.display import clear_output
from utils.eval import get_ap_ar, get_ap_ar_for_train_val
from utils.train import get_optimiser, get_lr_scheduler, print_params_setup
from utils.init import reproducibility, clean_memory_get_device
from models.dynamic_loss import DynamicWeightedLoss
from data.constants import DEFAULT_REFLACX_LABEL_COLS, XAMI_MIMIC_PATH
from  datetime import datetime
import torch.optim as optim

## Suppress the assignement warning from pandas.r
pd.options.mode.chained_assignment = None  # default='warn'

## Supress user warning
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

%matplotlib inline

In [3]:
device = clean_memory_get_device()
reproducibility()


This notebook will running on device: [CPU]


In [7]:
use_iobb = True
io_type_str = "IoBB" if use_iobb else "IoU"
labels_cols = DEFAULT_REFLACX_LABEL_COLS
iou_thrs = np.array([0.5])

common_args = {
    "save_early_stop_model": True,
    "optimiser": "sgd",
    "lr": 1e-3,
    "weight_decay": 1e-5,
    "image_backbone_pretrained": True,
    "fixation_backbone_pretrained": True,
    "record_training_performance": True,
    "dataset_mode": "normal",
    "image_size": 512,
    "batch_size": 4,
    "warmup_epochs": 0,
    "lr_scheduler": "ReduceLROnPlateau",
    "reduceLROnPlateau_factor": 0.1,
    "reduceLROnPlateau_patience": 999,
    "reduceLROnPlateau_full_stop": True,
    "multiStepLR_milestones": 100,
    "multiStepLR_gamma": 0.1,
    "use_mask": True,
    "gt_in_train_till": 999,
    "box_head_dropout_rate": 0,
    "measure_test": True,
}

fusion_add_args = {
    "fuse_depth": 0,
    "fusion_residule": False,
    "fusion_strategy": "add", 
}

small_model_args = {
    "mask_hidden_layers": 64,
    "fuse_conv_channels": 64,
    "representation_size": 64, 
    "backbone_out_channels": 64,
}

mobilenet_args = {
    "backbone": "mobilenet_v3",
    "using_fpn": False,
}

In [9]:
num_epochs = 100
train_infos = []

all_model_setups = [
    ModelSetup(
        name="forward_testing_model",
        use_fixations=True,
        **mobilenet_args,
        **small_model_args,
        **common_args,
        **fusion_add_args,
    )
]

for model_setup in all_model_setups:

    print_f.print_title("Preparing for the training.")

    train_info = TrainingInfo(model_setup)

    if model_setup.measure_test:
        # initialise the test recording list.
        train_info.test_ap_ars = []

    model = create_multimodal_rcnn_model(
            labels_cols,
            model_setup,
            rpn_nms_thresh=0.3,
            box_detections_per_img=10,
            box_nms_thresh=0.2,
            rpn_score_thresh=0.0,
            box_score_thresh=0.05,
    )

    model.to(device)

    ################ Datasets ################
    dataset_params_dict = {
            "XAMI_MIMIC_PATH": XAMI_MIMIC_PATH,
            "dataset_mode": model_setup.dataset_mode,
            "bbox_to_mask": model_setup.use_mask,
            "labels_cols": DEFAULT_REFLACX_LABEL_COLS,
    }

    detect_eval_dataset, train_dataset, val_dataset, test_dataset = get_datasets(
            dataset_params_dict=dataset_params_dict
    )

    train_dataloader, val_dataloader, test_dataloader = get_dataloaders(
            train_dataset, val_dataset, test_dataset, batch_size=2
    )

    train_coco, val_coco, test_coco = get_cocos(
            train_dataloader, val_dataloader, test_dataloader
    )

#     eval_params_dict = get_eval_params_dict(
#             detect_eval_dataset, iou_thrs=iou_thrs, use_iobb=use_iobb,
#     )

    # dynamic_loss_weight = None
    loss_keys = [
            "loss_classifier",
            "loss_box_reg",
            "loss_objectness",
            "loss_rpn_box_reg",
    ]
            
    dynamic_loss_weight = DynamicWeightedLoss(
            keys=loss_keys + ["loss_mask"] if model_setup.use_mask else loss_keys
    )
    dynamic_loss_weight.to(device)
    print_params_setup(model)

    params = [p for p in model.parameters() if p.requires_grad]
    if dynamic_loss_weight:
        params += [p for p in dynamic_loss_weight.parameters() if p.requires_grad]

    iou_types = get_iou_types(model, model_setup)
    optimizer = get_optimiser(params, model_setup)
    lr_scheduler = get_lr_scheduler(optimizer, model_setup)

    current_time = datetime.now()

    print_f.print_title(
        f"Start training. Preparing Tooke [{ (current_time - train_info.start_t).seconds}] sec"
    )

    train_info.start_t = datetime.now()

    val_loss = None

    ## Start the training from here.
    for e in range(num_epochs):

        print_f.print_title(f"Training model: [{model_setup.name}]")
        print(train_info)

        train_info.epoch = e + 1

        if train_info.epoch > model_setup.gt_in_train_till:
            model.roi_heads.use_gt_in_train  = False

        ###### Perform training and show the training result here ######
        model.train()

        train_info.last_train_evaluator, train_loger = xami_train_one_epoch(
                model=model,
                optimizer=optimizer,
                data_loader=train_dataloader,
                device=device,
                epoch=train_info.epoch,
                print_freq=10,
                iou_types=iou_types,
                coco=train_coco,
                score_thres=None,
                evaluate_on_run=False,
                # params_dict=eval_params_dict,
                dynamic_loss_weight=dynamic_loss_weight,
        )

        # train_info.train_evaluators.append(train_evaluator)
        train_info.train_losses.append(get_data_from_metric_logger(train_loger))
        ################################################################


====================Preparing for the training.====================
Using pretrained backbone. mobilenet_v3
Using pretrained backbone. mobilenet_v3
forward_testing_model will use mask, [64] layers.
creating index...
index created!
creating index...
index created!
creating index...
index created!
[model]: 4,749,149
[model.backbone]: 1,258,848
[model.rpn]: 41,803
[model.roi_heads]: 2,189,650
[model.roi_heads.box_head]: 204,928
[model.roi_heads.box_head.fc6]: 200,768
[model.roi_heads.box_head.fc7]: 4,160
[model.roi_heads.box_predictor]: 975
[model.roi_heads.mask_head]: 1,917,952
Using SGD as optimizer with lr=0.001
====================Start training. Preparing Tooke [28] sec====================
====================Training model: [forward_testing_model]====================
========================================For Training [forward_testing_model]========================================
ModelSetup(dataset_mode='normal', name='forward_testing_model', use_fixations=True, save_early_stop_mo

TypeError: cannot unpack non-iterable MetricLogger object